In [1]:
from qcodes import InstrumentChannel, Instrument, ChannelList
from qcodes import ManualParameter, MultiParameter, ArrayParameter
from qcodes.utils.validators import Numbers
import numpy as np

User schema at /Users/jhn/qcodesrc_schema.json not found.User settings won't be validated


In [2]:
class DummyChannel(InstrumentChannel):
    """
    A single dummy channel implementation
    """

    def __init__(self, parent, name, channel):
        super().__init__(parent, name)

        self._channel = channel

        # Add the various channel parameters
        self.add_parameter('temperature',
                           parameter_class=ManualParameter,
                           initial_value=0,
                           label="Temperature_{}".format(channel),
                           unit='K',
                           vals=Numbers(0, 300))
        
        self.add_parameter(name='foobar',
                           parameter_class=MultiSetPointParam)
        
        self.add_parameter(name='myarray',
                   parameter_class=ArraySetPointParam)


class DummyChannelInstrument(Instrument):
    """
    Dummy instrument with channels
    """

    def __init__(self, name, **kwargs):
        super().__init__(name, **kwargs)

        channels = ChannelList(self, "TempSensors", DummyChannel, snapshotable=False)
        for chan_name in ('A', 'B', 'C', 'D', 'E', 'F'):
            channel = DummyChannel(self, 'Chan{}'.format(chan_name), chan_name)
            channels.append(channel)
            self.add_submodule(chan_name, channel)
        self.add_submodule("channels", channels)

In [3]:
class MultiSetPointParam(MultiParameter):
    """
    Multiparameter which only purpose it to test that units, setpoints
    and so on are copied correctly to the individual arrays in the datarray.
    """
    def __init__(self, instrument, name):
        name = 'testparameter'
        shapes = ((5,), (5,))
        names = ('this', 'that')
        labels = ('this label', 'that label')
        units = ('this unit', 'that unit')
        sp_base = tuple(np.linspace(5, 9, 5))
        setpoints = ((sp_base,), (sp_base,))
        setpoint_names = (('this_setpoint',), ('this_setpoint',))
        setpoint_labels = (('this setpoint',), ('this setpoint',))
        setpoint_units = (('this setpointunit',), ('this setpointunit',))
        super().__init__(name, names, shapes,
                         labels=labels,
                         units=units,
                         setpoints=setpoints,
                         setpoint_labels=setpoint_labels,
                         setpoint_names=setpoint_names,
                         setpoint_units=setpoint_units)

    def get(self):
        return np.zeros(5), np.ones(5)

In [4]:
class ArraySetPointParam(ArrayParameter):
    """
    Multiparameter which only purpose it to test that units, setpoints
    and so on are copied correctly to the individual arrays in the datarray.
    """
    def __init__(self, instrument, name):
        name = 'testparameter'
        shape = (5,)
        label = 'this label'
        unit = 'this unit'
        sp_base = tuple(np.linspace(5, 9, 5))
        setpoints = (sp_base,)
        setpoint_names = ('this_setpoint',)
        setpoint_labels = ('this setpoint',)
        setpoint_units = ('this setpointunit',)
        super().__init__(name,
                         shape,
                         label=label,
                         unit=unit,
                         setpoints=setpoints,
                         setpoint_labels=setpoint_labels,
                         setpoint_names=setpoint_names,
                         setpoint_units=setpoint_units)

    def get(self):
        return np.random.rand(5)

In [5]:
inst = DummyChannelInstrument('foobar')

In [6]:
inst.channels[0]

<DummyChannel: foobar_ChanA of DummyChannelInstrument: foobar>

In [7]:
from qcodes import Loop

In [8]:
data1 = Loop(inst.A.temperature.sweep(0,10,1), 1).each(inst.A.foobar).run()

Started at 2017-06-06 17:11:20
DataSet:
   location = 'data/2017-06-06/#020_{name}_17-11-20'
   <Type>   | <array_id>                   | <array.name>  | <array.shape>
   Setpoint | foobar_ChanA_temperature_set | temperature   | (11,)
   Setpoint | this_setpoint_set            | this_setpoint | (11, 5)
   Measured | this                         | this          | (11, 5)
   Measured | that                         | that          | (11, 5)
Finished at 2017-06-06 17:11:31


In [9]:
data1.this

DataArray[11,5]: this
array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [10]:
data2 = Loop(inst.A.temperature.sweep(0,10,1), 1).each(inst.channels[0].foobar).run()

Started at 2017-06-06 17:11:33
DataSet:
   location = 'data/2017-06-06/#021_{name}_17-11-33'
   <Type>   | <array_id>                   | <array.name>  | <array.shape>
   Setpoint | foobar_ChanA_temperature_set | temperature   | (11,)
   Setpoint | this_setpoint_set            | this_setpoint | (11, 5)
   Measured | this                         | this          | (11, 5)
   Measured | that                         | that          | (11, 5)
Finished at 2017-06-06 17:11:44


In [11]:
data2 = Loop(inst.A.temperature.sweep(0,10,1), 1).each(inst.channels[0:1].foobar).run()

AttributeError: 'MultiSetPointParam' object has no attribute 'label'

In [13]:
data3 = Loop(inst.A.temperature.sweep(0,10,1), 1).each(inst.A.myarray).run()

Started at 2017-06-06 17:13:16
DataSet:
   location = 'data/2017-06-06/#023_{name}_17-13-16'
   <Type>   | <array_id>                   | <array.name>  | <array.shape>
   Setpoint | foobar_ChanA_temperature_set | temperature   | (11,)
   Measured | testparameter                | testparameter | (11, 5)
Finished at 2017-06-06 17:13:27


In [14]:
data3 = Loop(inst.A.temperature.sweep(0,10,1), 1).each(inst.channels[0].myarray).run()

Started at 2017-06-06 17:14:09
DataSet:
   location = 'data/2017-06-06/#024_{name}_17-14-09'
   <Type>   | <array_id>                   | <array.name>  | <array.shape>
   Setpoint | foobar_ChanA_temperature_set | temperature   | (11,)
   Measured | testparameter                | testparameter | (11, 5)
Finished at 2017-06-06 17:14:20


In [15]:
data3 = Loop(inst.A.temperature.sweep(0,10,1), 1).each(inst.channels[0:1].myarray).run()

Started at 2017-06-06 17:14:32
DataSet:
   location = 'data/2017-06-06/#025_{name}_17-14-32'
   <Type>   | <array_id>                   | <array.name>         | <array.shape>
   Setpoint | foobar_ChanA_temperature_set | temperature          | (11,)
   Measured | foobar_ChanA_myarray         | foobar_ChanA_myarray | (11,)
Finished at 2017-06-06 17:14:33


ValueError: setting an array element with a sequence.

In [17]:
inst.channels.myarray.get()

(array([ 0.16087586,  0.63407137,  0.40978536,  0.76905476,  0.82876632]),
 array([ 0.69882021,  0.73425924,  0.25545122,  0.86773728,  0.10338811]),
 array([ 0.8698955 ,  0.3354928 ,  0.07489469,  0.38043035,  0.70433896]),
 array([ 0.45961196,  0.92813073,  0.5977839 ,  0.35455607,  0.06801803]),
 array([ 0.49382892,  0.90160562,  0.36905395,  0.33856894,  0.39967096]),
 array([ 0.8050116 ,  0.74691103,  0.65995307,  0.85211923,  0.57228292]))

In [18]:
inst.channels.foobar.get()

AttributeError: 'MultiSetPointParam' object has no attribute 'label'

In [19]:
inst.channels.myarray.labels

('this label',
 'this label',
 'this label',
 'this label',
 'this label',
 'this label')

In [20]:
inst.A.myarray.label

'this label'

In [21]:
inst.A.myarray.shape

(5,)

In [22]:
inst.channels.myarray.shapes

((), (), (), (), (), ())